In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 2.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=adf0b4885329da62e9b7fe97ed4865496d541f110f3b1eccfc2644198f41d7c8
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [ ]:
!pip install gensim

In [ ]:
#Basic Libararies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle, islice
import string
import re

#Tensorflow, sklearn and nltk
import tensorflow as tf
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
#!pip install unidecode
#import unidecode
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from tensorflow.keras.layers import TextVectorization
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Flatten, Dropout, MaxPool2D, GlobalAvgPool2D, GlobalMaxPool2D
from tensorflow.keras.layers import GlobalAvgPool1D, GlobalMaxPool1D
from sklearn.model_selection import train_test_split
#!pip install tensorflow-addons
#import tensorflow_addons as tfa

#WordCloud
from wordcloud import WordCloud

#gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#Spacy
import spacy

#Vis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

#warnings
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
Path = '/content/gdrive/MyDrive/Capstone/vendors-cves.csv'

In [ ]:
df = pd.read_csv(Path)
df

,Unnamed: 0,_id,id,assigner,Published,Modified,last-modified,summary,access,impact,...,Unnamed: 7063,Unnamed: 7064,Unnamed: 7065,Unnamed: 7066,Unnamed: 7067,Unnamed: 7068,Unnamed: 7069,Unnamed: 7070,Unnamed: 7071,Unnamed: 7072
0,0,6211d7cc1291c8f278e33f0f,CVE-2008-0027,psirt@cisco.com,2008-01-17 3:00,2018-10-15 21:57,2018-10-15 21:57,Heap-based buffer overflow in the Certificate ...,"{'authentication': 'NONE', 'complexity': 'LOW'...","{'availability': 'COMPLETE', 'confidentiality'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,6211d7cccaa888fa42e33eec,CVE-2008-0028,psirt@cisco.com,2008-01-23 21:00,2018-10-26 14:19,2018-10-26 14:19,Unspecified vulnerability in Cisco PIX 500 Ser...,"{'authentication': 'NONE', 'complexity': 'MEDI...","{'availability': 'COMPLETE', 'confidentiality'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,6211d7cccaa888fa42e33f46,CVE-1999-0158,cve@mitre.org,1998-08-31 4:00,2018-10-30 16:25,2018-10-30 16:25,Cisco PIX firewall manager (PFM) on Windows NT...,"{'authentication': 'NONE', 'complexity': 'LOW'...","{'availability': 'NONE', 'confidentiality': 'P...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,6211d7cccaa888fa42e33f64,CVE-1999-0293,cve@mitre.org,1998-01-01 5:00,2008-09-09 12:34,2008-09-09 12:34,AAA authentication on Cisco systems allows att...,"{'authentication': 'NONE', 'complexity': 'LOW'...","{'availability': 'PARTIAL', 'confidentiality':...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,6211d7cccaa888fa42e33f6a,CVE-2009-0057,psirt@cisco.com,2009-01-22 18:30,2017-08-08 1:33,2017-08-08 1:33,The Certificate Authority Proxy Function (CAPF...,"{'authentication': 'NONE', 'complexity': 'MEDI...","{'availability': 'PARTIAL', 'confidentiality':...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5874,5487,6211d8a0caa888fa42e365d0,CVE-2020-3468,psirt@cisco.com,2020-07-16 18:15,2020-07-23 17:07,2020-07-23 17:07,A vulnerability in the web-based management in...,"{'authentication': 'SINGLE', 'complexity': 'LO...","{'availability': 'NONE', 'confidentiality': 'P...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5875,5488,6211d8a0caa888fa42e365dd,CVE-2020-3470,psirt@cisco.com,2020-11-18 19:15,2021-08-06 19:06,2021-08-06 19:06,Multiple vulnerabilities in the API subsystem ...,"{'authentication': 'NONE', 'complexity': 'LOW'...","{'availability': 'COMPLETE', 'confidentiality'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5876,5489,6211d9401980d5b6dee36ad7,CVE-2021-44228,security@apache.org,2021-12-10 10:15,2022-02-19 4:23,2022-02-19 4:23,Apache Log4j2 2.0-beta9 through 2.15.0 (exclud...,"{'authentication': 'NONE', 'complexity': 'MEDI...","{'availability': 'COMPLETE', 'confidentiality'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5877,oller_supervisor:1.5\\(3h\\)1:*:*:*:*:*:*:*','cpe:2.3:a:cisco:integrated_management_contro...,'cpe:2.3:a:cisco:integrated_management_contro...,'cpe:2.3:a:cisco:integrated_management_contro...,'cpe:2.3:a:cisco:integrated_management_contro...,'cpe:2.3:a:cisco:integrated_management_contro...,'cpe:2.3:a:cisco:integrated_management_contro...,'cpe:2.3:a:cisco:integrated_management_contro...,'cpe:2.3:a:cisco:integrated_management_contro...,'cpe:2.3:a:cisco:integrated_management_contro...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(df.summary)
print (lemmatized_texts[0][0:90])

base buffer overflow provider service ctlprovider.exe allow remote attacker cause denial s


In [ ]:
lemmatized_texts[0:5]

['base buffer overflow provider service ctlprovider.exe allow remote attacker cause denial service execute arbitrary code long request',
 'unspecified vulnerability when live decrement feature enable allow remote attacker cause denial service device reload craft ip packet',
 'firewall manager allow attacker connect port server retrieve file name location know',
 'aaa authentication system allow attacker execute command authorization',
 'service 6.1(3 allow remote attacker cause denial service voice service outage send malformed input session client terminate prematurely']

## YAKE

In [ ]:
!pip install yake
import yake

     |████████████████████████████████| 60 kB 2.5 MB/s 
     |████████████████████████████████| 132 kB 8.9 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73995 sha256=d033a0a548fa2bcdcad40cc414e34c54584fd10d11211124d905c96c8ab821a0
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built jellyfish


In [ ]:
def lines(n):
    t=[]
    for i in range(n):
        t.append(lemmatized_texts[i])
    for i in t:
        d={}
        language = "en"
        max_ngram_size = 5
        deduplication_threshold = 0.9
        numOfKeywords = 6
        custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
        keywords = custom_kw_extractor.extract_keywords(i)
        for kw in keywords:
            d[kw[0]]=kw[1]    
        print(min(d.items(),key=lambda x:x[1]))

In [ ]:
lines(2)

('execute arbitrary code long request', 0.00012953480683582157)
('denial service device reload craft', 0.004357108274714684)


## GENSIM

In [ ]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['base', 'buffer', 'overflow', 'provider', 'service', 'ctlprovider', 'exe', 'allow', 'remote', 'attacker', 'cause', 'denial', 'service', 'execute', 'arbitrary', 'code', 'long', 'request']


In [ ]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2)]
allow


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.209538  0.046491       1        1  51.333790
1      0.014258 -0.172718       2        1  28.732011
0     -0.223797  0.126227       3        1  19.934199, topic_info=              Term         Freq        Total Category  logprob  loglift
171            cpe  2190.000000  2190.000000  Default  30.0000  30.0000
27   vulnerability  9527.000000  9527.000000  Default  29.0000  29.0000
193          could  8759.000000  8759.000000  Default  28.0000  28.0000
5            cause  2123.000000  2123.000000  Default  27.0000  27.0000
753      condition  1294.000000  1294.000000  Default  26.0000  26.0000
..             ...          ...          ...      ...      ...      ...
15         request   238.914909  1192.405612   Topic3  -5.0618   0.0051
132           user   272.142038  2654.540057   Topic3  -4.9316  -0.6650
122           read   197.212702   497.490425   Topic3  -5.2537   0.6874
21         feature   189.745946   508.970470   Topic3  -5.2923   0.6260
126  authenticated   194.926645  1564.757383   Topic3  -5.2653  -0.4701

[215 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
106       1  0.731279    access
106       2  0.062381    access
106       3  0.206290    access
781       2  0.997198  adjacent
781       3  0.003392  adjacent
...     ...       ...       ...
28        1  0.155110      when
28        2  0.468776      when
28        3  0.375710      when
1041      2  0.989059      will
1041      3  0.014545      will

[326 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [ ]:
lda_model.print_topics()

[(0,
  '0.058*"cpe" + 0.041*"allow" + 0.038*"remote" + 0.034*"attacker" + 0.024*"device" + 0.023*"aka" + 0.016*"session" + 0.015*"service" + 0.014*"authentication" + 0.013*"firmware"'),
 (1,
  '0.063*"vulnerability" + 0.059*"could" + 0.055*"attacker" + 0.046*"device" + 0.039*"affected" + 0.038*"exploit" + 0.033*"allow" + 0.031*"cause" + 0.024*"condition" + 0.018*"service"'),
 (2,
  '0.063*"vulnerability" + 0.061*"attacker" + 0.057*"could" + 0.039*"allow" + 0.036*"exploit" + 0.027*"affected" + 0.025*"interface" + 0.025*"user" + 0.023*"web" + 0.019*"system"')]

In [ ]:
print("\nPerplexity:\t",lda_model.log_perplexity(corpus, total_docs=10000),'\n')

coherence_model_lda = CoherenceModel(model=lda_model, texts=df.summary, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print("\nCoherence Score:\t",coherence_lda,'\n')


Perplexity:	 -5.282490018613095 


Coherence Score:	 nan 

